In [1]:
!pip install -qq git+https://github.com/huggingface/transformers.git@main accelerate bitsandbytes

In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.float16
)

In [4]:
model_id = "codellama/CodeLlama-34b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    device_map="auto",
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=8192, out_features=22016, bias=False)
          (up_proj): Linear4bit(in_features=8192, out_features=22016, bias=False)
          (down_proj): Linear4bit(in_features=22016, out_features=8192, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [7]:
user = "I am using Python. How can I write the most optimal code to generate prime numbers till n?"

prompt = f"<s>[INST] {user.strip()} [/INST]"
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    add_special_tokens=False,
)

In [8]:
output = model.generate(
    inputs["input_ids"].to('cuda'),
    max_new_tokens=300,
    do_sample=True,
    top_p=0.9,
    temperature=0.1,
)
output = output[0].to("cpu")
decoded_outout = tokenizer.decode(output)
print(decoded_outout)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] I am using Python. How can I write the most optimal code to generate prime numbers till n? [/INST]  There are several ways to generate prime numbers in Python, and the most optimal approach depends on the value of `n`. Here are a few methods:

1. Sieve of Eratosthenes: This is a simple and efficient method for finding all prime numbers up to a given number `n`. The basic idea is to start with a list of all numbers up to `n` and then iteratively mark as composite (not prime) any number that is divisible by a prime number less than or equal to its square root. Here's an example implementation:
```
def sieve_of_eratosthenes(n):
    prime_list = [2]
    for i in range(3, n+1, 2):
        if i % 2 == 0:
            prime_list.append(i)
    for i in range(3, int(n**0.5)+1, 2):
        if prime_list[i]:
            for j in range(i*i, n+1, i):
                prime_list[j] = False
    return prime_list
```
2. Miller-Rabin primality test: This is a probabilistic primality test that 